In [3]:
from sklearn.tree import DecisionTreeClassifier
from matplotlib import pyplot
from sklearn.tree import plot_tree
from sklearn.metrics import confusion_matrix
from sklearn.metrics import recall_score

In [4]:
# Scikit-learn implementa el algoritmo CART de tal manera que, al buscar en
# cada nodo la mejor combinación atributo-umbral para particionar el conjunto
# de ejemplos asociado al nodo, no considera los atributos en orden, sino que
# los reordena aleatoriamente. Esto puede hacer que se obtenga un árbol
# distinto cada vez que se ejecuta el código, ya que produce como efecto que
# los empates en esa búsqueda se deshacen de forma aleatoria. Por tanto, para
# que el código sea reproducible debemos establecer una semilla aleatoria inicial.

# Por otra parte, para evitar el sobreajuste del árbol a los ejemplos de
# entrenamiento vamos a limitar la profundidad del árbol construido.

clasificador_CART = DecisionTreeClassifier(
    max_depth=10 #random_state=numero, en pricipio siemprecrea el mismo arbol. Retro & nuskool if you know what i mean
)
clasificador_CART.fit(atributos, objetivo) 

NameError: name 'atributos' is not defined

In [ ]:
pyplot.figure(figsize=(50, 20))  # Anchura y altura del gráfico
árbol = plot_tree(clasificador_CART,
                  # El argumento feature_names permite proporcionar, en una
                  # lista, los nombres de los atributos.
                  feature_names=atributos_discretos + atributos_continuos,
                  # El argumento class_names permite proporcionar, en una
                  # lista, los nombres de las clases
                  class_names=['No', 'Yes'])

In [ ]:
predicciones = clasificador_CART.predict(atributos)
predicciones

In [ ]:
clasificador_CART.score(atributos, objetivo)

In [ ]:
confusion_matrix(objetivo, predicciones)

In [ ]:
recall_score(objetivo, predicciones)

In [ ]:
(atributos_entrenamiento, atributos_prueba,
 objetivo_entrenamiento, objetivo_prueba) = train_test_split(
        # Conjuntos de datos a dividir, usando los mismos índices para ambos
        atributos, objetivo,
        # Tamaño del conjunto de prueba (20 % en este caso)
        test_size=.2,
        # Estratificación según la distribución de clases en el atributo objetivo
        stratify=objetivo)

In [ ]:
clasificador_CART = DecisionTreeClassifier(
    max_depth=100,  # Máxima profundidad del árbol
    min_samples_split=50  # Mínimo número de ejemplos para poder particionar
)

In [ ]:
resultados_validación_cruzada = cross_validate(clasificador_CART,
                                               atributos_entrenamiento,
                                               objetivo_entrenamiento,
                                               scoring='recall',
                                               cv=10)
resultados_validación_cruzada

In [ ]:
resultados_validación_cruzada['test_score'].mean()

In [ ]:
clasificador_CART = DecisionTreeClassifier()
rejilla_de_hiperparámetros = {
    # Máxima profundidad del árbol: 3, 4, 5, 6, 7, 8, 9, 10
    'max_depth': range(3, 11),
    # Mínimo número de ejemplos para poder particionar: 5, 10, 15
    'min_samples_split': range(5, 20, 5)
}

In [ ]:
CV(clasificador_CART,
                                   rejilla_de_hiperparámetros,
                                   scoring='recall',
                                   cv=10)
búsqueda_en_rejilla.fit(atributos_entrenamiento, objetivo_entrenamiento)búsqueda_en_rejilla.best_score_

In [ ]:
búsqueda_en_rejilla.best_params_